# Lesson 32: Python Advanced - Extension of a class

## Callable class instance

In [4]:
# Objects can be called as if they were a function:

class MemoryClass:
    
    def __init__(self, list):
        self.list_of_items = list
        
    # By adding this internal function of the class we make the instance callable
    def __call__(self, item):
        self.list_of_items.append(item)
        
# Calling such a class:

mem = MemoryClass([])
print("List of items in memory:", mem.list_of_items)

# Adding items to the instance of this class in the standard way (without using its feature of being callable):

mem.list_of_items.append("buy sugar")

# Adding items when the instance is callable (after definig a callable method __call__)

mem("buy coffe")

mem("buy tea")

print("List of items in memory:", mem.list_of_items)

# Checking if the class and the instance are callable:

print("This class is callable:", callable(MemoryClass))
print("This instance is callable:", callable(mem))

# Note that the instance is calledble only after definig __cal__ method in the class!


List of items in memory: []
List of items in memory: ['buy sugar', 'buy coffe', 'buy tea']
This class is callable: True
This instance is callable: True


## Class as a decorator of a function

In [10]:
# We have a car dealer:

import random

cars = ["Opel", "Mercedes", "BMW", "Toyota", "Suzuki", "Ford", "Dacia", "Volvo", "Fiat", "Renault", "Peugeot", 
        "Porsche", "Audi", "Mazda"]

# We randomly choose a car for promotion:

def SelectTodayPromotion(list_of_cars):
    return random.choice(list_of_cars)

# We randomly choose a car to be presented in a show:

def SelectTodayShow(list_of_cars):
    return random.choice(list_of_cars)

# We randomly choose a car to offer its accessories for free:

def SelectFreeAccessories(list_of_cars):
    return random.choice(list_of_cars)

print("Promotion:", SelectTodayPromotion(cars))
print("Show:", SelectTodayShow(cars))
print("Free accessories:", SelectFreeAccessories(cars))


# But we actually want that if one car was selected in first random selection, it cannot be chosen 
# in 2 other. To apply this limitation we will use the class defined before.

Promotion: Mercedes
Show: Dacia
Free accessories: Mazda


In [26]:


class MemoryClass:
    
    list_of_already_selected_items = []
    
    def __init__(self, funct):
        #print(">>This is init of MemoryClass")
        self.funct = funct
        
    def __call__(self, list):
        #print(">> this is call of MemoryClass instance")
        items_not_selected = [i for i in list if not i in MemoryClass.list_of_already_selected_items]
        #print("*----- selecting only from:", items_not_selected)
        item = self.funct(items_not_selected)
        MemoryClass.list_of_already_selected_items.append(item)
        return item
        
cars = ["Opel", "Mercedes", "BMW", "Toyota", "Suzuki", "Ford", "Dacia", "Volvo", "Fiat", "Renault", "Peugeot", 
        "Porsche", "Audi", "Mazda"]

@MemoryClass
def SelectTodayPromotion(list_of_cars):
    return random.choice(list_of_cars)

@MemoryClass
def SelectTodayShow(list_of_cars):
    return random.choice(list_of_cars)

@MemoryClass
def SelectFreeAccessories(list_of_cars):
    return random.choice(list_of_cars)

print("Promotion:", SelectTodayPromotion(cars))
print("Show:", SelectTodayShow(cars))
print("Free accessories:", SelectFreeAccessories(cars))

Promotion: Toyota
Show: Opel
Free accessories: Mazda


## Operators of the class

In [40]:
# To discuss this we will use the example of class shown in previous class:

class Car:
    
    def __init__(self, brand, model, isAirBagOk, isPaintingOk, isMechanicsOk, accessories):
        
        self.brand = brand
        self.model = model
        self.isAirBagOk = isAirBagOk
        self.isPaintingOk = isPaintingOk
        self.isMechanicsOk = isMechanicsOk
        self.accessories = accessories
    
    def GetInfo(self):
        print("-"*30)
        print("{} {}".format(self.brand, self.model).upper())
        print("Air Bag   -   ok  -  {}".format(self.isAirBagOk))
        print("Painting  -   ok  -  {}".format(self.isPaintingOk))
        print("Mechanics -   ok  -  {}".format(self.isMechanicsOk))
        print("Accessories       -  {}".format(self.accessories))
        print("-"*30)
        
    # Here we will use the operator "iadd". There are special internal predefined functions for the operators.
    # We want to have possibilities of offering new accessories:
    
    def __iadd__(self, other):
        if type(other) is list:
            accessories = self.accessories
            accessories.extend(other)
            return Car(self.brand, self.model, self.isAirBagOk, self.isPaintingOk, self.isMechanicsOk, accessories)
        elif type(other) is str:
            accessories = self.accessories
            accessories.append(other)
            return Car(self.brand, self.model, self.isAirBagOk, self.isPaintingOk, self.isMechanicsOk, accessories)
        else:
            raise Exception("Adding type {} to Car is not implemented".format(type(other)))
            
    # Now we need to have a method which works at the level of class and add 2 cars:
    
    def __add__(self, other):
        if type(other) is Car:
            return [self, other]
        else:
            raise Exception("Adding type {} to Car is not implemented".format(type(other)))
            
    # To convert obiect type to string it is convenient to define one more function:
    
    def __str__(self):
        return "Brand: {}, Model: {}".format(self.brand, self.model)


car_01 = Car("Seat", "Ibiza", True, True, True, ["winter tires"])

# We add the accessories by:

car_01 += ["navigation system", "roof rack"]   # This uses "if" in the class method __iadd__.

car_01 += "loadspeaker system"              # This is a string so it uses "elif" part in the class method __iadd__

car_01.GetInfo()

car_02 = Car("Opel", "Corsa", True, False, True, [])

car_pack = car_01 + car_02    # Here I use the function __add__

print("car_01+car_02:", car_pack[0].brand, car_pack[1].brand)

print(car_01)   # Here I use the function __str__

------------------------------
SEAT IBIZA
Air Bag   -   ok  -  True
Painting  -   ok  -  True
Mechanics -   ok  -  True
Accessories       -  ['winter tires', 'navigation system', 'roof rack', 'loadspeaker system']
------------------------------
car_01+car_02: Seat Opel
Brand: Seat, Model: Ibiza


## Inheritance

In [50]:
# Using the class Car created before we need to add some functions so that this class describes a truck.
# We do not want to modify the old class but we want to add "capacity" to its definition.

brandOnSale = "Opel"


# Before the class Car did not take in any argument. If I now add the argument "object" nothing will change for 
# the cars defined by this class. But if I define a new class "Truck", this "object" will allow me to inherit
# all features defined by the class "Car".
class Car:
    
    numberOfCars = 0
    listOfCars = []
    
    def __init__(self, brand, model, isAirBagOk, isPaintingOk, isMechanicsOk, __isOnSale):
        print(">> this is __init__ of the parent class:", self.__class__.__name__)
        
        self.brand = brand
        self.model = model
        self.isAirBagOk = isAirBagOk
        self.isPaintingOk = isPaintingOk
        self.isMechanicsOk = isMechanicsOk
        self.__isOnSale = __isOnSale
        
        Car.numberOfCars += 1
        Car.listOfCars.append(self)
        
    def IsDamaged(self):
        return not (self.isAirBagOk and self.isPaintingOk and self.isMechanicsOk)
    
    def GetInfo(self):
        print("-"*30)
        print("{} {}".format(self.brand, self.model).upper())
        print("Air Bag   -   ok  -  {}".format(self.isAirBagOk))
        print("Painting  -   ok  -  {}".format(self.isPaintingOk))
        print("Mechanics -   ok  -  {}".format(self.isMechanicsOk))
        print("IS ON SALE:          {}".format(self.__isOnSale))
        #print("-"*30)
        
    def __GetOnSale(self):
        return(self.__isOnSale)
    
    def __SetIsOnSale(self, newIsOnSaleStatus):
        if self.brand == brandOnSale:
            self.__isOnSale = newIsOnSaleStatus
            print("Changing status isOnSale to {} for {}".format(newIsOnSaleStatus, self.brand))
        else:
            print("Cannot change the status isOnSale. Sale valid only for {}".format(brandOnSale))
    
    IsOnSale = property(__GetOnSale, __SetIsOnSale, None, "if set to true, the car is available in sale")


class Truck(Car):
    def __init__(self, brand, model, isAirBagOk, isPaintingOk, isMechanicsOk, __isOnSale, capacityKg):
        print(">> this is __init__ of child class:", self.__class__.__name__)
        
        # Here I call the mother class to define attributes of the current class in a short way:
        # I need to use super().
        
        super().__init__(brand, model, isAirBagOk, isPaintingOk, isMechanicsOk, __isOnSale)
        
        # To call the mother class we can aslo use: (this is alternative way)
        # Car.__init__(self, brand, model, isAirBagOk, isPaintingOk, isMechanicsOk, __isOnSale)
        
        self.capacityKg = capacityKg
        
    # If I have new attributes if the class Truck (as capacityKg), then, to call methods which include 
    # all attributes I define a new method, which inherites the method from the mother class 
    # by using super(), and adds new functions:
    
    def GetInfo(self):
        super().GetInfo()
        print("CapacityKg:          {}".format(self.capacityKg))
        
  

truck_01 = Truck("Ford", "Transit", True, True, False, True, 1600)
truck_02 = Truck("Renault", "Trafic", True, False, True, True, 1200)
    
#car_01 = Car("Seat", "Ibiza", True, True, True, False)
#car_02 = Car("Opel", "Corsa", True, False, True, True)


# Calling properties of trucks:

print("Calling properties:", truck_01.brand, truck_01.capacityKg, truck_01.IsOnSale)

# Calling methods of trucks:

truck_01.GetInfo()

# Note that GetInfo() takes all info from the class Car and adds all info in the class Truck.

# Note that now both classes identify themselves as the parent class.

>> this is __init__ of child class: Truck
>> this is __init__ of the mother class: Truck
>> this is __init__ of child class: Truck
>> this is __init__ of the mother class: Truck
Calling properties: Ford 1600 True
------------------------------
FORD TRANSIT
Air Bag   -   ok  -  True
Painting  -   ok  -  True
Mechanics -   ok  -  False
IS ON SALE:          True
CapacityKg:          1600


## Inheritance from many classes

In [66]:
# Here we will use the old example but in a simplified version:

class Car:
    
    def __init__(self, brand, model, isOnSale):
        
        print(">> Class Car - init -starting")
        self.brand = brand
        self.model = model
        self.isOnSale = isOnSale
        self.name = "{} {}".format(brand, model)
        print(">> Class Car - init - finishing")
    
    def GetInfo(self):
        
        print(">> Class Car - GetInfo - starting")
        super().GetInfo()
        print("{} {}".format(self.brand, self.model).upper())
        print("IS ON SALE:          {}".format(self.isOnSale))
        print(">> Class Car - GetInfo - finishing")
        
# We define now another class Specialist (of one given brand):

class Specialist:
    
    def __init__(self, firstname, lastname, brand):
        
        print(">> Class Specialist - init -starting")
        self.firstname = firstname
        self.lastname = lastname
        # Note that we have the same attribute as in the class Car, but it means sth different. 
        # So there is a conflict.
        self.name = "{} {}".format(firstname, lastname)
        # Brand is also the same as in the class Car.
        self.brand = brand
        print(">> Class Specialist - init - finishing")
    
    def GetInfo(self):
        
        print(">> Class Specialist - GetInfo - starting")
        print("{} {} - ({})".format(self.firstname, self.lastname, self.brand))
        print(">> Class Specialist - GetInfo - finishing")


# We define a new class which will be inheriting from both classes above:

class CarSpecialist(Car, Specialist):
    
    # We need to convey all unique values that appear in Car and in Specialist:
    
    def __init__(self, brand, model, isOnSale, firstname, lastname):
        
        print(">> Class CarSpecialist - init - starting")
        # Here I cannot use super() to call the method of the parent class, because it will only take 
        # values from one class, and not 2. So I use the alternative method to call methods from the parent class:
        Car.__init__(self, brand, model, isOnSale)
        Specialist.__init__(self, firstname, lastname, brand.upper())
        print(">> Class CarSpecialist - init - finishing")
        
    def GetInfo(self):
        
        print(">> Class CarSpecialist - GetInfo - starting")
        super().GetInfo()
        print(">> Class CarSpecialist - GetInfo - finishing")
        
tom = CarSpecialist("Toyota", "Corolla", True, "Tom", "Smith")
print(vars(tom))

# Note that when some values appear in both classes, the first class creates them, but the second class
# overwrites them according to its own defined rules.

print("------"*10)

tom.GetInfo()

# Note that GetInfo() is started from CarSpecialist. There, in the definition it is declared that first parent
# is the Car class, and the second parent is the Specialist class. So, getting information is done in this order.

# If I rename GetInfo() in the class Car to any other name, it will be ignored in the execution of GetInfo()
# in CarSpecialist class. If I remove the line with super() in Car class, then only the rest of the method 
# will be executed, and GetInfo() from Specialist class will not be called at all.

# If super() is not used, only the first parent class is called. If it is used, it searches through
# other classes and calls those where the called method exists.
# This mechanism of placing super() in consequtive classes is called "method resolution order".

print("***"*20)

# MRO (method resolution order):
print(CarSpecialist.__mro__)


>> Class CarSpecialist - init - starting
>> Class Car - init -starting
>> Class Car - init - finishing
>> Class Specialist - init -starting
>> Class Specialist - init - finishing
>> Class CarSpecialist - init - finishing
{'brand': 'TOYOTA', 'model': 'Corolla', 'isOnSale': True, 'name': 'Tom Smith', 'firstname': 'Tom', 'lastname': 'Smith'}
------------------------------------------------------------
>> Class CarSpecialist - GetInfo - starting
>> Class Car - GetInfo - starting
>> Class Specialist - GetInfo - starting
Tom Smith - (TOYOTA)
>> Class Specialist - GetInfo - finishing
TOYOTA COROLLA
IS ON SALE:          True
>> Class Car - GetInfo - finishing
>> Class CarSpecialist - GetInfo - finishing
************************************************************
(<class '__main__.CarSpecialist'>, <class '__main__.Car'>, <class '__main__.Specialist'>, <class 'object'>)


## Class documenting

In [67]:
# How we should comment the code of a class:



class Car:
    """
    Car - class operating on cars available in the dealer
    """
    
    def __init__(self, brand, model, isAirBagOk, isPaintingOk, isMechanicsOk, __isOnSale):
        
        """
        init - arguments accepted:
        brand - the brand of the car is Fiat
        model - the model of the car is Multipla
        isAirBagOk - is the AirBag not used?
        isPaintingOk - is the paint of the car original/no corrections?
        isMechanicsOk - is the car free of any mechanics failure?
        __isOnSale - (hidden) is the car covered by some extra promotion (some additional conditions apply)?
        """
        
        self.brand = brand
        self.model = model
        self.isAirBagOk = isAirBagOk
        self.isPaintingOk = isPaintingOk
        self.isMechanicsOk = isMechanicsOk
        self.__isOnSale = __isOnSale
    
    @property
    def IsOnSale(self):
        """IsOnSale - the car is on extra promotion that is limited in time. Only selected cars can be on sale"""
        return self.__isOnSale
    
    @IsOnSale.setter
    def IsOnSale(self, newIsOnSaleStatus):
        if self.brand == brandOnSale:
            self.__isOnSale = newIsOnSaleStatus
            print("Changing status isOnSale to {} for {}".format(newIsOnSaleStatus, self.brand))
        else:
            print("Cannot change the status isOnSale. Sale valid only for {}".format(brandOnSale) )
            
    # If I do not define deleter here I could not delete a changed attribute.
    
    @IsOnSale.deleter
    def IsOnSale(self):
        self.__isOnSale = None
        
    # Additionally, I can define various types of property, here showing the text in a nice way:
    # Here, setter and deleter are not needed, but if needed they can be added to each function with @property.
    @property
    def CarTitle(self):
        return "Brand: {}, Model: {}".format(self.brand, self.model)
        
        
# The commnts are useful and I can call them:

help(Car)
help(Car.IsOnSale)

# new_car = Car()
# Shift+Tab give access to what should be written inside ().

Help on class Car in module __main__:

class Car(builtins.object)
 |  Car(brand, model, isAirBagOk, isPaintingOk, isMechanicsOk, _Car__isOnSale)
 |  
 |  Car - class operating on cars available in the dealer
 |  
 |  Methods defined here:
 |  
 |  __init__(self, brand, model, isAirBagOk, isPaintingOk, isMechanicsOk, _Car__isOnSale)
 |      init - arguments accepted:
 |      brand - the brand of the car is Fiat
 |      model - the model of the car is Multipla
 |      isAirBagOk - is the AirBag not used?
 |      isPaintingOk - is the paint of the car original/no corrections?
 |      isMechanicsOk - is the car free of any mechanics failure?
 |      __isOnSale - (hidden) is the car covered by some extra promotion (some additional conditions apply)?
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  CarTitle
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined 